In [7]:
import xgboost as xgb
import numpy as np

from scipy.io import loadmat, savemat

In [3]:
training_genuine_i_vectors = loadmat('./cqcc_i_vec_wcmvn/training_genuine_i_vectors.mat').get('training_genuine_i_vectors')
training_spoof_i_vectors = loadmat('./cqcc_i_vec_wcmvn/training_spoof_i_vectors.mat').get('training_spoof_i_vectors')
evaluation_i_vectors_ubm = loadmat('./cqcc_i_vec_wcmvn/evaluation_i_vectors_ubm.mat').get('evaluation_i_vectors_ubm')

In [4]:
train_data = np.concatenate([training_genuine_i_vectors, training_spoof_i_vectors], axis = 1).T
evaluation_i_vectors_ubm = evaluation_i_vectors_ubm.T
labels = np.concatenate([np.ones(training_genuine_i_vectors.shape[1]), np.zeros(training_spoof_i_vectors.shape[1])])

In [6]:
xgb_params = {
    'eta': 0.05,
    'gamma': 0.1,
    'lambda': 1,
    'alpha': 0,
    'max_depth': 3,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 0.86,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 1,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'nthread': 12,
    'seed': 1,
    'silent': 1
}
n_estimators = 2000

xgb_train = xgb.DMatrix(train_data, label=labels, missing=np.nan)
xgb_val = xgb.DMatrix(evaluation_i_vectors_ubm, missing=np.nan)


model = xgb.train(xgb_params, xgb_train, num_boost_round=n_estimators, verbose_eval=1)
y_pred = model.predict(xgb_val)

scores = {'prob': np.log(y_pred)}
savemat('scores.mat', scores)